In [70]:
import numpy as np

kontury = "dane\Kontury_eksport_dz.txt"

data = np.array([])
with open(kontury, mode='r') as file:
    for line in file:
        data = np.append(data, line.strip())

codes = []
i = 0
while (i+2) < len(data):
    code = data[i]
    num_of_lines = int(data[i+2])
    codes.append(code)
    i += num_of_lines + 4

possible_ofus = set(['B', 'Ba', 'Bi', 'Bp', 'Bz', 'K', 'dr', 'Tk', 'Ti', 'Tp', 'Wm', 'Wp', 'Ws', 'Tr', 'Ls', 'Lz', 'N', 'S', 'Br', 'Wsr', 'W', 'Lzr', 'R', 'Ps', 'Ł'])
unclassified = set(['B', 'Ba', 'Bi', 'Bp', 'Bz', 'K', 'dr', 'Tk', 'Ti', 'Tp', 'Wm', 'Wp', 'Ws', 'Tr', 'N'])
r_classes = set(['I', 'II', 'IIIa', 'IIIb', 'IVa', 'IVb', 'V', 'VI', 'VIz'])
lpslslz_classes = set(['I', 'II', 'III', 'IV', 'V', 'VI'])

In [71]:
def check_first_number(code, dash):
    for char in code[:dash]:
        if not char.isdigit():
            return 1
    return 0

def check_second_number(code, dash, slash):
    for char in code[slash:dash]:
        if not char.isdigit():
            return 1
    return 0

def check_first_type(code, slash_index):
    if code[slash_index+1:slash_index+4] in possible_ofus:
        return code[slash_index+1:slash_index+4]
    elif code[slash_index+1:slash_index+3] in possible_ofus:
        return code[slash_index+1:slash_index+3]
    elif code[slash_index+1:slash_index+2] in possible_ofus:
        return code[slash_index+1:slash_index+2]
    else:
        return None

def check_r(code, next_index):
    length = len(code)
    left = length - next_index
    if left == 0:
        print(f'Oznaczenie {code} powinno zawierać OZK dla R.')
        return 1
    elif left > 4:
        print(f'Oznaczenie {code} zawiera więcej znaków niż dopuszczalnych dla OZK dla R.')
        return 1
    else:
        if code[next_index:] in r_classes:
            return code[next_index:]
        print(f"Niepoprawne określenie OZK dla R dla {code}.")
        return 1

def check_lpslslz(code, next_index, uzytek_type):
    length = len(code)
    left = length - next_index
    if left == 0:
        print(f'Oznaczenie {code} powinno zawierać OZK dla {uzytek_type}.')
        return 1
    elif left > 3:
        print(f'Oznaczenie {code} zawiera więcej znaków niż dopuszczalnych dla OZK dla R.')
        return 1
    else:
        if code[next_index:] in lpslslz_classes:
            return code[next_index:]
        print(f"Niepoprawne określenie klasy dla {uzytek_type} dla {code}.")
        return 1

def check_sbrwsrwlzr(code, next_index, uzytek_type):
    length = len(code)
    left = length - next_index
    if left < 2:
        print(f'Oznaczenie {code} powinno zawierać OZU i OZK dla {uzytek_type}.')
        return 1
    elif code[next_index] != '-':
        print(f'Niepoprawne połączenie między OFU i OZU dla {code}.')
        return 1
    elif code[next_index + 1: next_index + 3] == 'Ps':
        if len(code) > next_index + 3:
            if code[next_index + 3:] not in lpslslz_classes:
                print(f'Niepoprawne OZK dla {code}.')
        else:
            print(f'Brak OZK dla {code}.')
    elif code[next_index + 1: next_index + 2] == 'Ł':
        if len(code) > next_index + 2:
            if code[next_index + 2:] not in lpslslz_classes:
                print(f'Niepoprawne OZK dla {code}.')
        else:
            print(f'Brak OZK dla {code}.')
    elif code[next_index + 1: next_index + 2] == 'R':
        if len(code) > next_index + 2:
            if code[next_index + 2:] not in r_classes:
                print(f'Niepoprawne OZK dla {code}.')
        else:
            print(f'Brak OZK dla {code}.')
    elif uzytek_type == 'W' and code[next_index + 1: next_index + 3] in ['Ls', 'Lz']:
        if len(code) > next_index + 3:
            if code[next_index + 3:] not in lpslslz_classes:
                print(f'Niepoprawne OZK dla {code}.')
        else:
            print(f'Brak OZK dla {code}.')
    else:
        print(f"Niepoprawna kombinacja OFU + OZU dla {code}.")
        
def check_poprawnosc(code, uzytek_type, slash_index):
    uzytek_len = len(uzytek_type)
    next_index = slash_index + uzytek_len + 1
    if uzytek_type in unclassified and len(code) > next_index:
        print(f'Ten typ użytku w kodzie {code} nie powinien zawierać żadnych znaków po {uzytek_type}.')
    elif uzytek_type == 'R':
        check_r(code, next_index)
    elif uzytek_type in ['Ł', 'Ps', 'Ls', 'Lz']:
        check_lpslslz(code, next_index, uzytek_type)
    elif uzytek_type in ['S', 'Br', 'Wsr', 'W', 'Lzr']:
        check_sbrwsrwlzr(code, next_index, uzytek_type)   


In [72]:
for code in codes:
    dash_index = code.find('-')
    slash_index = code.find('/')

    if dash_index == -1:
        print(f"Kod {code} nie zawiera dasha.")
        continue

    if slash_index == -1:
        print(f"Kod {code} nie zawiera slasha.")
        continue

    if check_first_number(code, dash_index) > 0:
        print(f"Błąd w pierwszej liczbie w kodzie {code}.")
        continue

    if check_second_number(code, dash_index, slash_index) > 0:
        print(f"Błąd w drugiej liczbie w kodzie {code}.")
        continue

    uzytek_type = check_first_type(code, slash_index)

    if uzytek_type is None:
        print(f"Niedozwolone oznaczenie w kodzie {code}.")
        continue

    check_poprawnosc(code, uzytek_type, slash_index)

Niepoprawne określenie klasy dla Ls dla 23-2/LsVIz.
Ten typ użytku w kodzie 23-3/B RIIIb nie powinien zawierać żadnych znaków po B.
Niepoprawne określenie klasy dla Ls dla 23-5/LsIVa.
Niepoprawne określenie klasy dla R dla 23-8/RIII.
Niepoprawne połączenie między OFU i OZU dla 23-12/W/RIVa.
Oznaczenie 23-14/Lzr powinno zawierać OZU i OZK dla Lzr.
Niedozwolone oznaczenie w kodzie 23-16/E-Wm.
Ten typ użytku w kodzie 23-19/N-RVI nie powinien zawierać żadnych znaków po N.
Niedozwolone oznaczenie w kodzie 23-20/E.
Niedozwolone oznaczenie w kodzie 23-22/E-N.
Niedozwolone oznaczenie w kodzie 23-24/E-LsV.
Niedozwolone oznaczenie w kodzie 23-27/E-PsIV.
Niedozwolone oznaczenie w kodzie 23-28/E-W.
Niedozwolone oznaczenie w kodzie 23-30/E-Lzr.
Niedozwolone oznaczenie w kodzie 23-31/E-Lzr-RVI.
Niepoprawne określenie klasy dla R dla 23-33/RIV.
Niepoprawna kombinacja OFU + OZU dla 23-34/S-N.
Oznaczenie 23-35/S powinno zawierać OZU i OZK dla S.
Niepoprawne określenie klasy dla Ps dla 23-37/PsIVa.
Ten 